In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import cvxpy as cp

# 1. 설정: 종목, 기간
tickers = ['SPY', 'QQQ', 'GLD', 'SCHD', 'TLT']
start_date = '2012-01-31'
end_date = '2025-04-30'

# 2. 데이터 다운로드 (월봉 종가)
data = yf.download(tickers, start=start_date, end=end_date, interval='1mo')['Close'].dropna()

# 3. 수익률 계산 (월간 수익률)
returns = data.pct_change().dropna()

# 4. 연환산 수익률 및 공분산 계산
# 월별 평균 수익률 * 12 (연환산), 공분산 행렬도 월별 데이터 기준 연환산
mu = returns.mean() * 12
S = returns.cov() * 12

# 5. base_weights 예시 (공격형 11번)
base_aggressive = {'SPY': 0.25, 'QQQ': 0.53, 'GLD': 0.22, 'SCHD': 0.0, 'TLT': 0.0}

def optimize_portfolio(base_weights, maximize="return", risk_limit=None):
    tickers_order = tickers
    w = cp.Variable(len(tickers_order))

    expected_ret = mu.values @ w
    portfolio_vol = cp.sqrt(cp.quad_form(w, S.values))

    if maximize == "return":
        objective = cp.Maximize(expected_ret)
    elif maximize == "sharpe":
        risk_free = 0.017  # 무위험 수익률
        objective = cp.Maximize((expected_ret - risk_free) / portfolio_vol)

    constraints = [cp.sum(w) == 1, w >= 0]

    # base_weights ± 0.1 범위 내 제한
    for i, t in enumerate(tickers_order):
        base = base_weights.get(t, 0)
        constraints += [w[i] >= max(0, base - 0.1), w[i] <= min(1, base + 0.1)]

    if risk_limit:
        constraints += [portfolio_vol <= risk_limit]

    problem = cp.Problem(objective, constraints)
    problem.solve()

    weights = np.round(w.value, 4)
    result = {
        'weights': dict(zip(tickers_order, weights)),
        'expected_return': expected_ret.value,
        'volatility': portfolio_vol.value,
        'sharpe': ((expected_ret.value - 0.017) / portfolio_vol.value)
    }
    return result

# 6. 최적화 실행 (예: 공격형 11번)
result = optimize_portfolio(base_aggressive, maximize="return")

print("최적화 결과:")
for k, v in result['weights'].items():
    print(f"{k}: {v:.4f}")
print(f"연 기대수익률: {result['expected_return']:.3f}")
print(f"연 변동성: {result['volatility']:.3f}")
print(f"Sharpe Ratio: {result['sharpe']:.3f}")

load D:\Project\Python_Source\Test01\.venv\Lib\site-packages\ortools\.libs\zlib1.dll...
load D:\Project\Python_Source\Test01\.venv\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load D:\Project\Python_Source\Test01\.venv\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load D:\Project\Python_Source\Test01\.venv\Lib\site-packages\ortools\.libs\re2.dll...
load D:\Project\Python_Source\Test01\.venv\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load D:\Project\Python_Source\Test01\.venv\Lib\site-packages\ortools\.libs\highs.dll...
load D:\Project\Python_Source\Test01\.venv\Lib\site-packages\ortools\.libs\ortools.dll...
(CVXPY) May 24 08:33:07 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.12.4544). Expected < 9.12.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) May 24 08:33:07 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version

[*********************100%***********************]  5 of 5 completed

최적화 결과:
SPY: 0.1500
QQQ: 0.6300
GLD: 0.1200
SCHD: 0.1000
TLT: 0.0000
연 기대수익률: 0.146
연 변동성: 0.139
Sharpe Ratio: 0.928


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import cvxpy as cp

# 1. 설정: 종목, 기간
tickers = ['SPY', 'QQQ', 'GLD', 'SCHD', 'TLT']
start_date = '2012-01-31'
end_date = '2025-04-30'

# 2. 데이터 다운로드 (월봉 종가)
data = yf.download(tickers, start=start_date, end=end_date, interval='1mo')['Close'].dropna()

# 3. 수익률 계산 (월간 수익률)
returns = data.pct_change().dropna()

# 4. 연환산 수익률 및 공분산 계산
mu = returns.mean() * 12
S = returns.cov() * 12

# 5. base_weights 예시 (공격형 11번)
base_aggressive = {'SPY': 0.25, 'QQQ': 0.53, 'GLD': 0.22, 'SCHD': 0.0, 'TLT': 0.0}

def optimize_portfolio(base_weights, maximize="return", risk_limit=None):
    tickers_order = tickers
    w = cp.Variable(len(tickers_order))

    expected_ret = mu.values @ w
    portfolio_vol = cp.sqrt(cp.quad_form(w, S.values))

    # 최적화 목표: 연수익률 최대화 or Sharpe Ratio 최대화
    if maximize == "return":
        objective = cp.Maximize(expected_ret)
    elif maximize == "sharpe":
        risk_free = 0.017  # 무위험 수익률
        objective = cp.Maximize((expected_ret - risk_free) / portfolio_vol)

    # 제약 조건
    constraints = [cp.sum(w) == 1, w >= 0]  # 비중 합 = 1, 음수 비중 불가

    # base_weights ± 0.1 범위 내 제한
    for i, t in enumerate(tickers_order):
        base = base_weights.get(t, 0)
        constraints += [w[i] >= max(0, base - 0.1), w[i] <= min(1, base + 0.1)]

    # 변동성 상한 제한 (선택사항)
    if risk_limit:
        constraints += [portfolio_vol <= risk_limit]

    # 최적화 수행
    problem = cp.Problem(objective, constraints)
    problem.solve()

    # 결과 정리
    weights = np.round(w.value, 4)
    result = {
        'weights': dict(zip(tickers_order, weights)),
        'expected_return': expected_ret.value,
        'volatility': portfolio_vol.value,
        'sharpe': ((expected_ret.value - 0.017) / portfolio_vol.value)
    }
    return result

# 6. 최적화 실행 (예: 공격형 11번, 연수익률 최대화)
result = optimize_portfolio(base_aggressive, maximize="return")

# 7. 결과 출력
print("\n📊 최적화 결과 (연수익률 최대화):")
for k, v in result['weights'].items():
    print(f"{k}: {v:.4f}")
print(f"\n연 기대수익률: {result['expected_return']:.3f}")
print(f"연 변동성: {result['volatility']:.3f}")
print(f"Sharpe Ratio: {result['sharpe']:.3f}")

[*********************100%***********************]  5 of 5 completed


📊 최적화 결과 (연수익률 최대화):
SPY: 0.1500
QQQ: 0.6300
GLD: 0.1200
SCHD: 0.1000
TLT: 0.0000

연 기대수익률: 0.146
연 변동성: 0.139
Sharpe Ratio: 0.928


In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
import cvxpy as cp

# 1. 설정: 종목, 기간
tickers = ['SPY', 'QQQ', 'GLD', 'SCHD', 'TLT']
start_date = '2012-01-31'
end_date = '2025-04-30'

# 2. 데이터 다운로드 (월봉 종가)
data = yf.download(tickers, start=start_date, end=end_date, interval='1mo')['Close'].dropna()

# 3. 수익률 계산 (월간 수익률)
returns = data.pct_change().dropna()

# 4. 연환산 수익률 및 공분산 계산
mu = returns.mean() * 12
S = returns.cov() * 12

# 5. base_weights (사용자 지정)
base_weights = {
    'SPY': 0.15,
    'QQQ': 0.63,
    'GLD': 0.12,
    'SCHD': 0.10,
    'TLT': 0.00
}

def optimize_portfolio_limited_adjustment(base_weights, maximize="return", risk_limit=None):
    tickers_order = tickers
    w = cp.Variable(len(tickers_order))

    expected_ret = mu.values @ w
    portfolio_vol = cp.sqrt(cp.quad_form(w, S.values))

    # 최적화 목표
    if maximize == "return":
        objective = cp.Maximize(expected_ret)
    elif maximize == "sharpe":
        risk_free = 0.017
        objective = cp.Maximize((expected_ret - risk_free) / portfolio_vol)

    # 제약 조건: base_weights ± 0.1 내에서만 조정
    constraints = [cp.sum(w) == 1, w >= 0]

    for i, t in enumerate(tickers_order):
        base = base_weights.get(t, 0)
        lower_bound = max(0, base - 0.1)
        upper_bound = min(1, base + 0.1)
        constraints += [w[i] >= lower_bound, w[i] <= upper_bound]

    if risk_limit:
        constraints += [portfolio_vol <= risk_limit]

    problem = cp.Problem(objective, constraints)
    problem.solve()

    weights = np.round(w.value, 4)
    result = {
        'weights': dict(zip(tickers_order, weights)),
        'expected_return': expected_ret.value,
        'volatility': portfolio_vol.value,
        'sharpe': ((expected_ret.value - 0.017) / portfolio_vol.value)
    }
    return result

# 6. 최적화 실행
result = optimize_portfolio_limited_adjustment(base_weights, maximize="return")

# 7. 결과 출력
print("\n📊 최적화 결과 (기대수익률 최대화, ±10% 제한):")
for k, v in result['weights'].items():
    print(f"{k}: {v:.4f}")
print(f"\n연 기대수익률: {result['expected_return']:.3f}")
print(f"연 변동성: {result['volatility']:.3f}")
print(f"Sharpe Ratio: {result['sharpe']:.3f}")

[*********************100%***********************]  5 of 5 completed


📊 최적화 결과 (기대수익률 최대화, ±10% 제한):
SPY: 0.0500
QQQ: 0.7300
GLD: 0.0200
SCHD: 0.2000
TLT: 0.0000

연 기대수익률: 0.161
연 변동성: 0.156
Sharpe Ratio: 0.924
